In [1]:
DATASET = 'dory'
KSIZE = 21
RADIUS = 1
PROJECT_ROOT = 'demo'
CDBG_DIR = f'{PROJECT_ROOT}/{DATASET}_k{KSIZE}'
CATLAS_DIR = f'{CDBG_DIR}_r{RADIUS}'
CDBG_PATH = f'{CDBG_DIR}/cdbg.gxt'
FIRST_DOMS_PATH = f'{CATLAS_DIR}/first_doms.txt'
CATLAS_PATH = f'{CATLAS_DIR}/catlas.csv'

In [2]:
import networkx as nx
from typing import List, Dict

In [3]:
def to_nx_graph(g):
    return nx.from_edgelist((u, v) for u, v, w in g.arcs())

In [4]:
def read_gxt(path: str) -> nx.Graph:
    """Reads SpaceGraphCats graph representation and converts it to a NetworkX graph.

    Args:
        path: path to the input file
    Return:
        NetworkX Graph instance        
    """
    G = nx.Graph()
    
    try:
        # Open and read file
        with open(path) as file:
    
            # Extract the number of vertices and add create each vertex
            num_vert = int(file.readline())            
            G = nx.empty_graph(num_vert)
        
            # Add an edge for each remaining line in the file
            for line in file:
                numbers = line.split()
                G.add_edge(int(numbers[0]), int(numbers[1]))
                
    except OSError:
        raise OSError('Error opening or closing the file.')
    except ValueError:
        raise ValueError('Unexpected contents in file.')
            
    return G

In [5]:
def read_partitions(path: str) -> Dict[int, List[int]]:
    """ Reads SpaceGraphCats lists of dominators and their neighbors. Returns a dictionary mapping each dominator to the vertices it dominates.
    
    Args:
        path: path to the input file
    Return:
        Dict[int List[int]]: maps dominators to the vertices it dominates.
    """
    dom_dict = {}
    
    try:
        # Open and read file
        with open(path) as file:
            for line in file:
                numbers = line.split()
                
                if (len(numbers) == 1):
                    raise ValueError('Unexpected contents in file.')
                
                part_list = []
                for i in range(1, len(numbers)):
                    part_list.append(int(numbers[i]))
                dom_dict[int(numbers[0])] = part_list
                
    except OSError:
        raise OSError('Error opening or closing the file.')
    except ValueError:
        raise ValueError('Unexpected contents in file.')
    
    return dom_dict

In [6]:
# load assignments
def load_assignments(level):
    if level == 1:
        # load 1st level partitioning
        first_doms = read_partitions(FIRST_DOMS_PATH)

        # use hash for more balanced colors (ideally, find proper coloring)
        return first_doms.keys(), {v: hash(str(d)) for d, vs in first_doms.items() for v in vs}
    else:
        doms = {}
        idx_to_vtx = {}

        with open(CATLAS_PATH) as f:
            for line in f:
                idx, v, lv, cld = line.split(',')
                idx, v, lv = map(int, [idx, v, lv])
                if lv == level:
                    doms[v] = map(int, cld.split())
                elif lv == level - 1:
                    idx_to_vtx[idx] = v

        return doms.keys(), {idx_to_vtx[i]: hash(str(d)) for d, vs in doms.items() for i in vs}

In [7]:
def write_atlas_level_to_gml(proj, level):
    file_name = f'{CATLAS_DIR}/level_{level}_atlas.gml'
    
    # load cDBG without isolates if level 0
    if level == 0:
        G0 = read_gxt(CDBG_PATH) # load the original cDBG
        isolates = [v for v in G0 if G0.degree(v) == 0] # find isolates
        G0.remove_nodes_from(isolates) # remove isolates
        G = G0
    
    # load level from checkpoint and generate graph
    else:
        proj.load_checkpoint(level)
        G = to_nx_graph(proj.graph)
        
    # write graph to gml
    nx.write_gml(G, file_name)

In [8]:
# load project
import spacegraphcats.catlas.catlas

proj = spacegraphcats.catlas.catlas.Project(CDBG_DIR, CATLAS_DIR, RADIUS)

In [9]:
write_atlas_level_to_gml(proj, 2)

Loading results of building level 2
